# ML Zoomcamp Homework 9

In [1]:
#Download model trained in Homework 8

#!wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/dino-dragon-model/dino_dragon_10_0.899.h5

## Question 1

In [2]:
#Import libraries
import numpy as np
import tensorflow as tf
from tensorflow import keras

2022-11-29 03:25:39.809247: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-29 03:25:39.996250: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-29 03:25:40.001889: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-29 03:25:40.001909: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [3]:
#Load downloaded model

model = keras.models.load_model("dino_dragon_10_0.899.h5")

#Convert model from Keras to TF-Lite format

converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

#Export model in TF-Lite format

with open("dino_dragon_model.tflite", "wb") as f_out:
    f_out.write(tflite_model)


2022-11-29 03:25:41.428649: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-29 03:25:41.428725: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-29 03:25:41.428743: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ef252fe8172d): /proc/driver/nvidia/version does not exist
2022-11-29 03:25:41.428937: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


INFO:tensorflow:Assets written to: /tmp/tmp0vntenmm/assets


INFO:tensorflow:Assets written to: /tmp/tmp0vntenmm/assets
2022-11-29 03:25:42.909515: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-11-29 03:25:42.909653: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2022-11-29 03:25:42.911137: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmp0vntenmm
2022-11-29 03:25:42.913945: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2022-11-29 03:25:42.914073: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmp0vntenmm
2022-11-29 03:25:42.918674: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:357] MLIR V1 optimization pass is not enabled
2022-11-29 03:25:42.920219: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2022-11-29 03:25:43.018914: I tensorflow/cc/saved_model/loader.cc:213] Running initializatio

In [4]:
#Obtain model file size

!ls -lh "dino_dragon_model.tflite"

-rw-r--r-- 1 jovyan users 43M Nov 29 03:25 dino_dragon_model.tflite


## Question 2

In [5]:
interpreter = tf.lite.Interpreter(model_path='dino_dragon_model.tflite')
interpreter.allocate_tensors()

#Obtain input details
input_index = interpreter.get_input_details()

#Obtain output details
output_index = interpreter.get_output_details()
output_index

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


[{'name': 'StatefulPartitionedCall:0',
  'index': 13,
  'shape': array([1, 1], dtype=int32),
  'shape_signature': array([-1,  1], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

## Question 3

### Pre-processing with keras

In [6]:
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [7]:
#Download image and resize
#!wget https://upload.wikimedia.org/wikipedia/commons/thumb/d/df/Smaug_par_David_Demaret.jpg/1280px-Smaug_par_David_Demaret.jpg
img = load_img('1280px-Smaug_par_David_Demaret.jpg', target_size=(150, 150))

In [8]:
#Convert image to np array and preprocess
x = np.array(img, dtype="float32")
X = np.array([x])
X = np.divide(X, 255)

print(f"The value in the first pixel is: {X[0,0,0,0]}")

The value in the first pixel is: 0.5529412031173706


### Pre-processing without keras

In [9]:
from io import BytesIO
from urllib import request
from PIL import Image

In [10]:
#Functions to download, resize and rescale images 

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img

def resize_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

def preprocess_image(img):
    x = np.array(img, dtype="float32")
    X = np.array([x])
    X = np.divide(X, 255)
    return X

In [11]:
#Download and pre-process image

url = "https://upload.wikimedia.org/wikipedia/commons/thumb/d/df/Smaug_par_David_Demaret.jpg/1280px-Smaug_par_David_Demaret.jpg"
img = download_image(url)
img = resize_image(img, (150,150))
X = preprocess_image(img)

print(f"The value in the first pixel is: {X[0,0,0,0]}")

The value in the first pixel is: 0.5529412031173706


/tmp/ipykernel_4620/2323172328.py:13: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  img = img.resize(target_size, Image.NEAREST)


## Question 4

In [12]:
#Apply original model to image

prediction = model.predict(X)
prediction

1/1 [==============================] - 0s 325ms/step


array([[0.8244861]], dtype=float32)

In [13]:
#Apply tflite model to image

interpreter.set_tensor(input_index[0]["index"], X)
interpreter.invoke()
prediction = interpreter.get_tensor(output_index[0]["index"])

prediction

array([[0.82448614]], dtype=float32)

# Question 5

**On terminal:**
* docker pull svizor42/zoomcamp-dino-dragon-lambda:v2
* docker images

# Question 6

In [14]:
def predict(url):
    img = download_image(url)
    img = resize_image(img, (150,150))
    X = preprocess_image(img)

    interpreter.set_tensor(input_index, X)
    interpreter.invoke()
    prediction = interpreter.get_tensor(output_index)
  
    return float(prediction[0,0])

def lambda_handler(event, context):
    url = event["url"]
    pred = predict(url)
    result = pred
    return result